In [ ]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [2]:
import subprocess
from pathlib import Path
from collections import Counter

In [ ]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

In [5]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8000/v1',
    '--api-key', 'xxx',
    #'--model-name', 'unsloth/gemma-3-4b-it',
    '--model-name', 'google/gemma-3-4b-it',
    '--output-jsonl', OUT_JSONL,
    #'--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--input-hf-dataset', 'Nitral-AI/Reddit-NSFW-Writing_Prompts_ShareGPT',
    
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "unsloth/gemma-3-4b-it",
    '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name google/gemma-3-4b-it --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-NSFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 40 --max-prompts 40 --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 2000 --request-mode chunk --chunk-size 20 --chat-template-model-id unsloth/gemma-3-4b-it --tdpo-pairs-jsonl tdpo-pairs.jsonl
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   1%|          | 39/5938 [00:00<00:00, 37812.73prompt/s]
Preparing to process 40 new prompts in this run.
BANNED (prompt_idx=16, type=slop_phrase, phrase='elias')                   
BANNED (prompt_idx=18, type=slop_phrase, phrase='scent')                  
BANNED (prompt_idx=23, type=slop_phrase, phrase='relentless')             
BANN

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


BANNED (prompt_idx=10, type=ngram, ngram='eyes fixed')                   
BANNED (prompt_idx=8, type=slop_phrase, phrase='nodded')                 
BANNED (prompt_idx=20, type=slop_phrase, phrase='sensation')             
BANNED (prompt_idx=26, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not as a solution, but ')
BANNED (prompt_idx=15, type=slop_phrase, phrase='smile spread across')   
BANNED (prompt_idx=6, type=slop_phrase, phrase='whisper')                
BANNED (prompt_idx=16, type=slop_phrase, phrase='elias')                 
BANNED (prompt_idx=25, type=ngram, ngram='voice laced')                  
BANNED (prompt_idx=38, type=slop_phrase, phrase='stared')                
BANNED (prompt_idx=17, type=slop_phrase, phrase='surge')                 
BANNED (prompt_idx=30, type=ngram, ngram='closed eyes')                  
BANNED (prompt_idx=1, type=slop_phrase, phrase='whisper')                
BANNED (prompt_idx=

Token indices sequence length is longer than the specified maximum sequence length for this model (1588 > 256). Running this sequence through the model will result in indexing errors


BANNED (prompt_idx=23, type=slop_phrase, phrase='chaotic')               
BANNED (prompt_idx=35, type=slop_phrase, phrase='fabric')                
BANNED (prompt_idx=6, type=slop_phrase, phrase='echo')                   
BANNED (prompt_idx=28, type=ngram, ngram='said voice')                   
BANNED (prompt_idx=3, type=slop_phrase, phrase='shifted')                
BANNED (prompt_idx=14, type=slop_phrase, phrase='vanished')              
BANNED (prompt_idx=20, type=slop_phrase, phrase='laced')                 
BANNED (prompt_idx=13, type=slop_phrase, phrase='murmured')              
BANNED (prompt_idx=1, type=slop_phrase, phrase='gaze')                   
BANNED (prompt_idx=22, type=slop_phrase, phrase='chuckled')              
BANNED (prompt_idx=25, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not enough to cause pain, but ')
BANNED (prompt_idx=12, type=slop_phrase, phrase='etched')                
BANNED (prom

CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'google/gemma-3-4b-it', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-NSFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '40', '--max-prompts', '40', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '2000', '--request-mode', 'chunk', '--chunk-size', '20', '--chat-template-model-id', 'unsloth/gemma-3-4b-it', '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'], returncode=0)